<a href="https://colab.research.google.com/github/kechcole/Choropleth-Maps-Using-Python/blob/main/Data_Engineering_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install arcpy

ERROR: Could not find a version that satisfies the requirement arcpy (from versions: none)
ERROR: No matching distribution found for arcpy


# **DATA ENGINEERING WITH PYTHON **



# 1. Loading and Preparing modules and data.

In [9]:
# Import modules ]

import pandas as pd
import os

In [10]:
# Upload files from directory

from google.colab import files
uploaded = files.upload()

Saving CountyCVAP_2006-2010.csv to CountyCVAP_2006-2010.csv
Saving CountyCVAP_2010-2014.csv to CountyCVAP_2010-2014.csv
Saving CountyCVAP_2014-2018.csv to CountyCVAP_2014-2018.csv
Saving CountyCVAP_2017-2021.csv to CountyCVAP_2017-2021.csv
Saving countypres_2000-2020.csv to countypres_2000-2020.csv


In [11]:
# Read data

elections_data_path = "countypres_2000-2020.csv"

# Create dataframe with county_fips attribute as an object
elections_complete_df = pd.read_csv(elections_data_path,  dtype={"county_fips":object})



# 2. Explore data.

In [12]:
# Head
elections_complete_df.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,RALPH NADER,GREEN,160,17208,20220315,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,OTHER,OTHER,113,17208,20220315,TOTAL
4,2000,ALABAMA,AL,BALDWIN,01003,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL


In [16]:
print("No of columns and rows : ", (elections_complete_df.shape))
print("No of columns : ", len(elections_complete_df))

No of columns and rows :  (72617, 12)
No of columns :  72617


In [20]:
# Data types

elections_complete_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72617 entries, 0 to 72616
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            72617 non-null  int64 
 1   state           72617 non-null  object
 2   state_po        72617 non-null  object
 3   county_name     72617 non-null  object
 4   county_fips     72560 non-null  object
 5   office          72617 non-null  object
 6   candidate       72617 non-null  object
 7   party           72617 non-null  object
 8   candidatevotes  72617 non-null  int64 
 9   totalvotes      72617 non-null  int64 
 10  version         72617 non-null  int64 
 11  mode            72617 non-null  object
dtypes: int64(4), object(8)
memory usage: 6.6+ MB


In [ ]:
# Basic ststistics

elections_complete_df.describe()

In [ ]:
# Rename columns to better identify them

rename_cols = {
    "state_po": "state_abbr",
    "county_fips": "FIPS",
    "party": "pol_identity"
}

elections_complete_df.rename(columns=rename_cols, inplace=True)
elections_complete_df.head()

# 2.1 Handling missing values

In [ ]:
# Count rows with Missing data in each column
# You will later join this table to voting-age population data based on the FIPS value,
# so this value cannot be null. You will identify how many rows have null values and
# establish a strategy for dealing with those null values.

elections_complete_df.isnull().sum()

In [ ]:
# Find records with null values in the FIPS column.

elections_complete_df.query("FIPS.isnull()")

In [24]:
# Identify the rows in Washington(DC) state and fill FIPS values with its code 11001

elections_complete_df.loc[ elections_complete_df["state_abbr"] == "DC", "FIPS"] = "11001"



In [28]:
# The remaining records with null FIPS values are votes assigned at a state-wide level,
# not to any county. Your analysis will take place at the county level, which means that
# these records cannot be used. You will assess the effect that removing these votes will
# have on your overall analysis.

# Count the number of rows
rowcount = elections_complete_df.shape[0]

# The total number of votes cast in all counties
votecount = elections_complete_df['candidatevotes'].sum()


# Determine how many votes are associated with null FIPS values, create a dataframe of missing values
null_fips_df = elections_complete_df.query("FIPS.isnull()")    # Record with null values
null_fips_rowcount = null_fips_df.shape[0]                     # Count records with null values
null_fips_votecount = null_fips_df["candidatevotes"].sum()     # Number of vote with missing data in FIPS column


# Calculate the percentage of the data affected by null records.
percentage_null_fips_rows = round((null_fips_rowcount / rowcount) * 100, 3)
percentage_null_fips_votes = round((null_fips_votecount / votecount) * 100, 3)


# Use a print statement to report this information.
print(f"There were {null_fips_rowcount} records with null FIPS values in the data, representing {null_fips_votecount} votes.")
print(f"This amounts to {percentage_null_fips_rows}% of the total records and {percentage_null_fips_votes}% of the total votes.")


There were 52 records with null FIPS values in the data, representing 13009 votes.
This amounts to 0.072% of the total records and 0.002% of the total votes.


From the observation above, the null values will not affect our study. We can remove these values.


In [29]:
# Remove missing values and create a new dataframe.

elections_df = elections_complete_df.query("FIPS.notnull()")
elections_df


,year,state,state_abbr,county_name,FIPS,office,candidate,pol_identity,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,RALPH NADER,GREEN,160,17208,20220315,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,OTHER,OTHER,113,17208,20220315,TOTAL
4,2000,ALABAMA,AL,BALDWIN,01003,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL
...,...,...,...,...,...,...,...,...,...,...,...,...
72612,2020,WYOMING,WY,WASHAKIE,56043,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,3245,4032,20220315,TOTAL
72613,2020,WYOMING,WY,WESTON,56045,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,360,3560,20220315,TOTAL
72614,2020,WYOMING,WY,WESTON,56045,US PRESIDENT,JO JORGENSEN,LIBERTARIAN,46,3560,20220315,TOTAL
72615,2020,WYOMING,WY,WESTON,56045,US PRESIDENT,OTHER,OTHER,47,3560,20220315,TOTAL


# **3. Pivoting the table.**

Even after correcting for the missing FIPS values, the format of the election data table will prevent a proper join to the voting-age population data. Currently, each record in the table corresponds to a candidate's votes for each county for each vote mode. You need to reformat the table so that each record corresponds to a single county for each election year, with fields showing the total votes for each major party candidate for that election year in that county.


The steps in restructuring the table are as follows:
1. Limit records to only candidates who identify as either a Democrat or a Republican
2. Sum each candidate's votes for every mode in each county, for each election year
3. Lock some fields in place, and then pivot the table using the party field.
4. Reformat the pivoted table.

In [30]:
# Filter records by political identity for Democrats and Republicans.

elections_df = elections_df.query("pol_identity in ['DEMOCRAT', 'REPUBLICAN']")
elections_df

,year,state,state_abbr,county_name,FIPS,office,candidate,pol_identity,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,01001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL
4,2000,ALABAMA,AL,BALDWIN,01003,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL
5,2000,ALABAMA,AL,BALDWIN,01003,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,40872,56480,20220315,TOTAL
8,2000,ALABAMA,AL,BARBOUR,01005,US PRESIDENT,AL GORE,DEMOCRAT,5188,10395,20220315,TOTAL
...,...,...,...,...,...,...,...,...,...,...,...,...
72608,2020,WYOMING,WY,UINTA,56041,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,7496,9459,20220315,TOTAL
72609,2020,WYOMING,WY,WASHAKIE,56043,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,651,4032,20220315,TOTAL
72612,2020,WYOMING,WY,WASHAKIE,56043,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,3245,4032,20220315,TOTAL
72613,2020,WYOMING,WY,WESTON,56045,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,360,3560,20220315,TOTAL


In [34]:
# Group records for each candidate by county and year.
# This splits the datasets into different groups
candidate_group = elections_df.groupby(['FIPS', 'county_name', 'state', 'candidate', 'year', 'pol_identity', 'totalvotes'])


# Sum each candidate's votes in each county, for each election year.
candidate_votes = candidate_group['candidatevotes'].sum()
candidate_votes

# Remove the multi-index created by the group-by and sum, returning a new data frame.
candidate_votes_df = candidate_votes.reset_index()

# Check the output.
candidate_votes_df.head()

,FIPS,county_name,state,candidate,year,pol_identity,totalvotes,candidatevotes
0,01001,AUTAUGA,ALABAMA,AL GORE,2000,DEMOCRAT,17208,4942
1,01001,AUTAUGA,ALABAMA,BARACK OBAMA,2008,DEMOCRAT,23641,6093
2,01001,AUTAUGA,ALABAMA,BARACK OBAMA,2012,DEMOCRAT,23932,6363
3,01001,AUTAUGA,ALABAMA,DONALD J TRUMP,2020,REPUBLICAN,27770,19838
4,01001,AUTAUGA,ALABAMA,DONALD TRUMP,2016,REPUBLICAN,24973,18172


In [35]:
# Pivot the data frame.
# The index "locks" the fields that remain the same for each county.
# The columns determine which values will become new fields; in this case, the two values in the pol_identity column become two new fields.
# The values column determines which values will be reported for each of the new fields.
elections_pivot_df = candidate_votes_df.pivot(
    index=['year', 'FIPS', 'county_name', 'state', 'totalvotes'],
    columns=['pol_identity'],
    values=['candidatevotes']
)

# Check the output.
elections_pivot_df.head()

candidatevotes           
pol_identity                                    DEMOCRAT REPUBLICAN
year FIPS  county_name state   totalvotes                          
2000 01001 AUTAUGA     ALABAMA 17208                4942      11993
     01003 BALDWIN     ALABAMA 56480               13997      40872
     01005 BARBOUR     ALABAMA 10395                5188       5096
     01007 BIBB        ALABAMA 7101                 2710       4273
     01009 BLOUNT      ALABAMA 17973                4977      12667